# Advent of Code 2021 day 16

In [27]:
from collections import *
from itertools import *
from functools import *
from dataclasses import dataclass

from aocd.models import Puzzle
import numpy as np
import parse
from aocp import *

example: str = """"""
example_sol_a: int = None
example_sol_b: int = None


puzzle = Puzzle(year=2021, day=16)
raw_data = puzzle.input_data

In [2]:
raw_data

'4057231006FF2D2E1AD8025275E4EB45A9ED518E5F1AB4363C60084953FB09E008725772E8ECAC312F0C18025400D34F732333DCC8FCEDF7CFE504802B4B00426E1A129B86846441840193007E3041483E4008541F8490D4C01A89B0DE17280472FE937C8E6ECD2F0D63B0379AC72FF8CBC9CC01F4CCBE49777098D4169DE4BF2869DE6DACC015F005C401989D0423F0002111723AC289DED3E64401004B084F074BBECE829803D3A0D3AD51BD001D586B2BEAFFE0F1CC80267F005E54D254C272950F00119264DA7E9A3E9FE6BB2C564F5376A49625534C01B0004222B41D8A80008446A8990880010A83518A12B01A48C0639A0178060059801C404F990128AE007801002803AB1801A0030A280184026AA8014C01C9B005CE0011AB00304800694BE2612E00A45C97CC3C7C4020A600433253F696A7E74B54DE46F395EC5E2009C9FF91689D6F3005AC0119AF4698E4E2713B2609C7E92F57D2CB1CE0600063925CFE736DE04625CC6A2B71050055793B4679F08CA725CDCA1F4792CCB566494D8F4C69808010494499E469C289BA7B9E2720152EC0130004320FC1D8420008647E8230726FDFED6E6A401564EBA6002FD3417350D7C28400C8C8600A5003EB22413BED673AB8EC95ED0CE5D480285C00372755E11CCFB164920070B40118DB1AE5901C0199DCD8D616CFA89009BF6008800

In [140]:
def parse_input(raw_data: str):
    return "".join([bin(int(c, 16))[2:].zfill(4) for c in raw_data])

In [139]:
example_data = parse_input(example)
data = parse_input(raw_data)

## Part 1

In [143]:
Packet = namedtuple("Packet", ["version", "type", "length", "packets", "literal"])

def parse_packet(data: str, offset: int = 0):
    string = data[offset:]
    version = int(string[:3], 2)
    packet_type = int(string[3:6], 2)
    packets = []    
    if packet_type == 4:
        i = 6
        literals = []
        is_last = False
        while not is_last:
            group = string[i:i+5]
            literals.append(group[1:])
            is_last = True if group[0] == "0" else False
            i += 5
        literal = int("".join(literals), 2)
        length = len(literals)*5+6
        return Packet(version, packet_type, length, packets, literal)
    else:
        total_length, num_packets, start = (0, int(string[7:18], 2), 18) if int(string[6:7], 2) else (int(string[7:22], 2), 0, 22)
        parsed_len = 0    
        while len(packets) < num_packets or parsed_len < total_length:
            packet = parse_packet(string, start+parsed_len)
            packets.append(packet)
            parsed_len += packet.length
        return Packet(version, packet_type, start+parsed_len, packets, 0)

In [144]:
parse_packet("00111000000000000110111101000101001010010001001000000000", 0)

Packet(version=1, type=6, length=49, packets=[Packet(version=6, type=4, length=11, packets=[], literal=10), Packet(version=2, type=4, length=16, packets=[], literal=20)], literal=0)

In [145]:
parse_packet("11101110000000001101010000001100100000100011000001100000", 0)

Packet(version=7, type=3, length=51, packets=[Packet(version=2, type=4, length=11, packets=[], literal=1), Packet(version=4, type=4, length=11, packets=[], literal=2), Packet(version=1, type=4, length=11, packets=[], literal=3)], literal=0)

In [146]:
def sum_versions(packet: Packet):
    return packet.version + sum(sum_versions(p) for p in packet.packets)

In [147]:
def solve_a(data) -> int:
    return sum_versions(parse_packet(data))

In [148]:
solution_a = solve_a(data)
print(solution_a)

996


In [137]:
puzzle.answer_a = solution_a

## Part 2

In [128]:
def operate_packet(packet: Packet):
    if packet.type == 4:
        return packet.literal
    elif packet.type == 0:
        return sum([operate_packet(p) for p in packet.packets])
    elif packet.type == 1:
        return np.prod([operate_packet(p) for p in packet.packets])
    elif packet.type == 2:
        return min([operate_packet(p) for p in packet.packets])
    elif packet.type == 3:
        return max([operate_packet(p) for p in packet.packets])
    elif packet.type == 5:
        return operate_packet(packet.packets[0]) > operate_packet(packet.packets[1])
    elif packet.type == 6:
        return operate_packet(packet.packets[0]) < operate_packet(packet.packets[1])
    elif packet.type == 7:
        return operate_packet(packet.packets[0]) == operate_packet(packet.packets[1])

In [129]:
def solve_b(data) -> int:
    return operate_packet(parse_packet(data))

In [130]:
solution_b = solve_b(data)
print(solution_b)

96257984154


In [118]:
puzzle.answer_b = solution_b

Part b already solved with same answer: 96257984154
